In [32]:
'''
Notebook for the main text's FIGS6: Influence of plasmid copy number fluctuations on the Punisher's performance.
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

cpu


/tmp/ipykernel_8251/935770666.py:21: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)


Loading BokehJS ...

In [33]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_cnc_b_initialise,
    circuits.punisher_cnc_b_ode,
    circuits.punisher_cnc_b_F_calc,
    circuits.punisher_cnc_b_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [34]:
# PARAMETERISE THE CIRCUIT

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
par['a_cat'] = 500.0  # promoter strength (unitless)
par['n_cat'] = 300.0
# synthetic protease gene
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# plasmid copy number control
init_conds['cat_pb'] = 10.0  # INITIAL CONDITION (not a parameter): all plasmids have working CAT gene copies
par['k_tr'] = 130.2 # plasmid replication rate (1/h)
par['a_inh'] = 948 # inhibitor synthesis rate per plasmid copy (1/h)
par['b_inh'] = 74.976 # inhibitor degradation rate (1/h)
par['n_inh'] = 10 # number of steps of replication initiation at which inhibition can happen
par['K_inh'] = 214.05 # replication inhibition constant (nM)

# l_cruise=1.34
# steady_inh_cruise=par['a_inh']*10/(par['b_inh']+l_cruise)
# par['K_inh']=steady_inh_cruise*1/((par['k_tr']/l_cruise)**(1/par['n_inh'])-1)
# print(steady_inh_cruise,par['K_inh'])

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [35]:
# SPECIFY WHICH CASE WE SIMULATE

# A, B, C - the Punisher still reacts to burdensome gene mutation
# case='burd_react'
# changing_plasmid_conc=False
# new_func_b=0.0
# new_plasmid_conc=0.0    # this value will be of no consequence

# D, E, F - even twofold concentration increases don't trigger the Punisher
# case='double_plasmid_conc'
# changing_plasmid_conc=True
# new_plasmid_conc=20.0
# new_func_b=1.0  # don't disable the burdensome gene

# G, H, I - even being left with just one Punisher copy does not do much
case='one_plasmid_copy'
changing_plasmid_conc=True
new_plasmid_conc=1.0
new_func_b=1.0  # don't disable the burdensome gene

In [36]:
# SET UP THE DETERMINISTIC SIMULATION PARAMETERS

# diffrax simulator
savetimestep = 0.1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

# simulation time frames - add half of the save time step to the end to include the last time point
tf_nopun = (0,25+savetimestep/2)
tf_pun = (25,50+savetimestep/2) # time frame for simulation before burdensome gene loss
tf_afterloss = (50,100+savetimestep/2) # time frame for simulation after burdensome gene loss

In [37]:
# RUN THE DETERMINISTIC SIMULATION

# initial simulation to get the steady state without gene expression loss
p_switch_ac_frac = par['p_switch_ac_frac']  # active fraction of protein (i.e. share of molecules bound by the inducer)
par['p_switch_ac_frac'] = 0.0  # set the active fraction to zero to get the steady state without the punisher
sol=ode_sim(par,    # dictionary with model parameters
            ode_with_circuit,   #  ODE function for the cell with synthetic circuit
            cellmodel_auxil.x0_from_init_conds(init_conds,circuit_genes,circuit_miscs),  # initial condition VECTOR
            len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
            cellmodel_auxil.synth_gene_params_for_jax(par,circuit_genes), # synthetic gene parameters for calculating k values
            tf_nopun, jnp.arange(tf_nopun[0], tf_nopun[1], savetimestep), # time frame and time axis for saving the system's state
            rtol, atol)    # relative and absolute tolerances
ts_nopun=np.array(sol.ts)
xs_nopun=np.array(sol.ys)

# simulation with the punisher
par['p_switch_ac_frac'] = p_switch_ac_frac  # set the active fraction back to the original value
sol=ode_sim(par,    # dictionary with model parameters
            ode_with_circuit,   #  ODE function for the cell with synthetic circuit
            sol.ys[-1,:],  # initial condition VECTOR
            len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
            cellmodel_auxil.synth_gene_params_for_jax(par,circuit_genes), # synthetic gene parameters for calculating k values
            tf_pun, jnp.arange(tf_pun[0], tf_pun[1], savetimestep), # time frame and time axis for saving the system's state
            rtol, atol)    # relative and absolute tolerances
ts_preloss=np.concatenate((ts_nopun,np.array(sol.ts)),axis=0)
xs_preloss=np.concatenate((xs_nopun,np.array(sol.ys)),axis=0)

# simulating synthetic gene expression loss
if(changing_plasmid_conc):
    x0_afterloss=sol.ys[-1,:].at[circuit_name2pos['cat_pb']].set(new_plasmid_conc)  # simulation will resume from the last time point, with an amended plamsid_conc
else:
    x0_afterloss=sol.ys[-1,:]   # simulation will resume from the last time point
par['func_b'] = new_func_b
sol=ode_sim(par,    # dictionary with model parameters
            ode_with_circuit,   #  ODE function for the cell with synthetic circuit
            x0_afterloss,  # initial condition VECTOR
            len(circuit_genes), len(circuit_miscs), circuit_name2pos, # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
            cellmodel_auxil.synth_gene_params_for_jax(par,circuit_genes), # synthetic gene parameters for calculating k values
            tf_afterloss, jnp.arange(tf_afterloss[0], tf_afterloss[1], savetimestep), # time frame and time axis for saving the system's state
            rtol, atol)    # relative and absolute tolerances
ts=np.concatenate((ts_preloss,np.array(sol.ts)),axis=0)
xs=np.concatenate((xs_preloss,np.array(sol.ys)),axis=0)

In [38]:
# PLOT THE CELL MODEL TRAJECTORIES

# cell protein mass breakdown
mass_fig=cellmodel_auxil.plot_protein_masses(ts,xs,par,circuit_genes) 

# native mRNA, tRNA & protein levels, chloramphenicol level
nat_mrna_fig,nat_prot_fig,nat_trna_fig,h_fig = cellmodel_auxil.plot_native_concentrations(ts, xs, par, circuit_genes)  

# cell growth, translation elongation, ribosome synth. regulation, ppGpp level, tRNA aminoacylation, Resource Competition Denominator
l_figure, e_figure, Fr_figure, ppGpp_figure, nu_figure, D_figure = cellmodel_auxil.plot_phys_variables(ts, xs, par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                                                       circuit_eff_m_het_div_k_het)  

# show plots
bkplot.show(bklayouts.grid([[mass_fig, nat_mrna_fig, nat_prot_fig],
                            [nat_trna_fig, h_fig, l_figure],
                            [e_figure, Fr_figure, D_figure]]))

In [39]:
# PLOT THE CIRCUIT TRAJECTORIES

# synthetic mRNA and protein levels, different states of CAT gene DNA
het_mrna_fig, het_prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations(ts, xs, par, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles)

# synthetic gene regulation
F_fig = cellmodel_auxil.plot_circuit_regulation(ts, xs, circuit_F_calc, par, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles)

misc_fig.y_range = bkmodels.Range1d(-0.5,12.5)
misc_fig.legend.location = "left"

# show plots
bkplot.show(bklayouts.grid([[het_mrna_fig, het_prot_fig, misc_fig],
                            [F_fig, None, None]]))


In [40]:
# MAKE FIGURE A, D OR G FOR THE PAPER

# get the growth rate for plotting
_, ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)

# determine axis labels and ranges
if(case=='burd_react'):
    x_axis_label="Time since mutation, h"
    x_range=(-5, 35)
else:
    x_axis_label="Time since plasmid copy\nnumber change, h"
    x_range=(-1, 7)
fig_adg_y_range=(0.5, 1.75)
# initialise
fig_adg = bkplot.figure(
    frame_width=150,
    frame_height=100,
    x_axis_label=x_axis_label,
    y_axis_label="Cell growth rate, 1/h",
    x_range=x_range,
    y_range=fig_adg_y_range,
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_adg.output_backend = "svg"

# if simulating burdensome gene mutation
if(case=='burd_react'):
    # add shading to show when synthetic gene expression loss occurs
    fig_adg.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[0,2,2,0],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_adg.add_layout(bkmodels.Label(x=0, y=fig_adg_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))
else:
    # mark the perturbation time point
    fig_adg.add_layout(bkmodels.Span(location=0, dimension='height', 
                                      line_color=bkRGB(0,0,0), line_width=1, line_dash='dashed'))
    if(new_plasmid_conc==1.0):
        copies_str='copy'
    else:
        copies_str='copies'
    fig_adg.add_layout(bkmodels.Label(x=0, y=fig_adg_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Set to '+str(int(new_plasmid_conc))+' plasmid '+copies_str,
                                    text_font_size='8pt'))

# plot the growth rate
fig_adg.line(ts-tf_afterloss[0],np.array(ls), line_width=2, line_color=bkRGB(0,0,0))

# set fonts
fig_adg.xaxis.axis_label_text_font_size = "8pt"
fig_adg.xaxis.major_label_text_font_size = "8pt"
fig_adg.yaxis.axis_label_text_font_size = "8pt"
fig_adg.yaxis.major_label_text_font_size = "8pt"

# set ticks
fig_adg.yaxis.ticker=np.arange(0.0, 1.76, 0.25)

# show plot
bkplot.show(fig_adg)

In [41]:
# MAKE FIGURE B, D OR H FOR THE PAPER

# y range for the plot (in terms of cat prot. conc.)
if(case=='burd_react'):
    x_axis_label="Time since mutation, h"
    x_range=(-5, 35)
else:
    x_axis_label="Time since plasmid copy\nnumber change, h"
    x_range=(-1, 7)
fig_bdh_y_range = (0, 1.25 * max(np.array(xs[:, circuit_name2pos['p_cat']])))
# initialise
fig_bdh = bkplot.figure(
    frame_width=150,
    frame_height=100,
    x_axis_label=x_axis_label,
    y_axis_label="CAT protein conc., nM",
    x_range=x_range,
    y_range=fig_bdh_y_range,
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_bdh.output_backend = "svg"

# if simulating burdensome gene mutation
if(case=='burd_react'):
    # add shading to show when synthetic gene expression loss occurs
    fig_bdh.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[0,fig_bdh_y_range[1],fig_bdh_y_range[1],0],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_bdh.add_layout(bkmodels.Label(x=0, y=fig_bdh_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))
else:
    # mark the perturbation time point
    fig_bdh.add_layout(bkmodels.Span(location=0, dimension='height', 
                                      line_color=bkRGB(0,0,0), line_width=1, line_dash='dashed'))
    if(new_plasmid_conc==1.0):
        copies_str='copy'
    else:
        copies_str='copies'
    fig_bdh.add_layout(bkmodels.Label(x=0, y=fig_bdh_y_range[1],
                                x_offset=2, y_offset=-16,
                                text='Set to '+str(int(new_plasmid_conc))+' plasmid '+copies_str,
                                text_font_size='8pt'))

# settings for the main y-axis (for p_cat)
fig_bdh.yaxis.axis_line_color=bkRGB(222, 49, 99)
fig_bdh.yaxis.major_tick_line_color=bkRGB(222, 49, 99)
fig_bdh.yaxis.minor_tick_line_color=bkRGB(222, 49, 99)

# plot the cat protein concentrations
fig_bdh.line(ts-tf_afterloss[0],xs[:,circuit_name2pos['p_cat']], line_width=2, line_color=bkRGB(222, 49, 99), legend_label="CAT")

# create an extra  y range for plotting integrase protein concentrations
fig_bdh.extra_y_ranges = {"p_int": bkmodels.Range1d(start=0, end=1.25 * np.max(xs[:, circuit_name2pos['p_int']]))}
fig_bdh.add_layout(bkmodels.LinearAxis(y_range_name="p_int",
                                     axis_label="Integrase conc., nM",
                                     axis_line_color=bkRGB(255, 103, 0),
                                     major_tick_line_color=bkRGB(255, 103, 0),
                                     minor_tick_line_color=bkRGB(255, 103, 0)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
fig_bdh.line(ts-tf_afterloss[0],xs[:,circuit_name2pos['p_int']], line_width=2, line_color=bkRGB(255, 103, 0), y_range_name="p_int", legend_label="Integrase")

# add legend
if(case=='burd_react' or case=='one_plasmid_copy'):
    fig_bdh.legend.location = "right"
else:
    fig_bdh.legend.location = "bottom_right"
fig_bdh.legend.label_text_font_size="8pt"
fig_bdh.legend.background_fill_alpha = 1
fig_bdh.legend.margin = 2
fig_bdh.legend.padding = 2
fig_bdh.legend.spacing = 2
fig_bdh.legend.glyph_width = 10

# set fonts
fig_bdh.xaxis.axis_label_text_font_size = "8pt"
fig_bdh.xaxis.major_label_text_font_size = "8pt"
fig_bdh.yaxis.axis_label_text_font_size = "8pt"
fig_bdh.yaxis.major_label_text_font_size = "8pt"

# show plots
bkplot.show(fig_bdh)

In [42]:
# MAKE FIGURE C, F OR I FOR THE PAPER

# get the growth rate for plotting
_, ls, _, _, _, _, _, _ =cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                       circuit_eff_m_het_div_k_het)

# determine axis labels and ranges
if(case=='burd_react'):
    x_axis_label="Time since mutation, h"
    x_range=(-5, 35)
else:
    x_axis_label="Time since plasmid copy\nnumber change, h"
    x_range=(-1, 7)
if(case=='burd_react' or case=='one_plasmid_copy'):
    fig_cfi_y_range=(0, 12.5)
else:
    fig_cfi_y_range=(0, 25)
# initialise
fig_cfi = bkplot.figure(
    frame_width=150,
    frame_height=100,
    x_axis_label=x_axis_label,
    y_axis_label="Plasmid conc., nM",
    x_range=x_range,
    y_range=fig_cfi_y_range,
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig_cfi.output_backend = "svg"

# if simulating burdensome gene mutation
if(case=='burd_react'):
    # add shading to show when synthetic gene expression loss occurs
    fig_cfi.add_layout(bkmodels.PolyAnnotation(xs=[0,0,tf_afterloss[1]-tf_afterloss[0],tf_afterloss[1]-tf_afterloss[0]],
                                             ys=[0,fig_cfi_y_range[1],fig_cfi_y_range[1],0],
                                             line_width=0, line_alpha=0,
                                             fill_color=bkRGB(100, 100, 100, 0.25)))
    fig_cfi.add_layout(bkmodels.Label(x=0, y=fig_cfi_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Burdensome gene mutated',
                                    text_font_size='8pt'))
else:
    # mark the perturbation time point
    fig_cfi.add_layout(bkmodels.Span(location=0, dimension='height', 
                                      line_color=bkRGB(0,0,0), line_width=1, line_dash='dashed'))
    if(new_plasmid_conc==1.0):
        copies_str='copy'
    else:
        copies_str='copies'
    fig_cfi.add_layout(bkmodels.Label(x=0, y=fig_cfi_y_range[1],
                                    x_offset=2, y_offset=-16,
                                    text='Set to '+str(int(new_plasmid_conc))+' plasmid '+copies_str,
                                    text_font_size='8pt'))

# plot the plasmid concentrations
fig_cfi.line(ts - tf_afterloss[0], xs[:, circuit_name2pos['cat_pb']], line_width=2,
             line_color=bkRGB(100, 149, 237), legend_label="ccat")
fig_cfi.line(ts - tf_afterloss[0], xs[:, circuit_name2pos['cat_lri1']], line_width=2,
             line_color=bkRGB(127, 255, 212), legend_label="cLRi")
fig_cfi.line(ts - tf_afterloss[0], xs[:, circuit_name2pos['no_cat']], line_width=2,
             line_color=bkRGB(207, 181, 59), legend_label="cno")

# legend settings
fig_cfi.legend.location = "bottom_right"
fig_cfi.legend.label_text_font_size="8pt"
fig_cfi.legend.background_fill_alpha = 1
fig_cfi.legend.margin = 5
fig_cfi.legend.padding = 2
fig_cfi.legend.spacing = 5
fig_cfi.legend.glyph_width = 10
if(case=='double_plasmid_conc'):
    fig_cfi.legend.orientation = "horizontal"
    fig_cfi.legend.margin = 5

# set fonts
fig_cfi.xaxis.axis_label_text_font_size = "8pt"
fig_cfi.xaxis.major_label_text_font_size = "8pt"
fig_cfi.yaxis.axis_label_text_font_size = "8pt"
fig_cfi.yaxis.major_label_text_font_size = "8pt"

# set ticks
if(case=='burd_react' or case=='one_plasmid_copy'):
    fig_cfi.yaxis.ticker=np.arange(0, 13, 2.5)
else:
    fig_cfi.yaxis.ticker=np.arange(0, 26, 5)

# show plot
bkplot.show(fig_cfi)